In [1]:
from lexrankr import LexRank

In [2]:
import pandas as pd
import re

In [119]:
submission= pd.read_csv("extractive_sample_submission_v2.csv",encoding='utf8')

In [120]:
submission

,id,summary
0,500733466,추출요약1\n추출요약2\n추출요약3
1,500742482,추출요약1\n추출요약2\n추출요약3
2,500742484,추출요약1\n추출요약2\n추출요약3
3,504213810,추출요약1\n추출요약2\n추출요약3
4,505279620,추출요약1\n추출요약2\n추출요약3
...,...,...
9982,745338220,추출요약1\n추출요약2\n추출요약3
9983,745367988,추출요약1\n추출요약2\n추출요약3
9984,745368130,추출요약1\n추출요약2\n추출요약3
9985,745368136,추출요약1\n추출요약2\n추출요약3


In [121]:
len(submission)

9987

In [3]:
test = pd.read_csv("extractive_test_v2.csv",encoding='utf8')

In [4]:
test

,media,id,article_original
0,당진시대,500733466,['▲ 석문간척지 임차법인협의회가 한국농어촌공사 당진지사 앞에 공공비축벼 320t을...
1,당진시대,500742482,"['신 벌떼해장국이 손님들의 성원에 보답하고자 24시간 영업을 재개한다.', '또한..."
2,당진시대,500742484,"['1년 전과 비교해 당진지역 휘발유 가격은 72원, 경유 가격은 103원 상승했다..."
3,당진시대,504213810,['어기구 국회의원이 천연가스의 안정적 수급을 위해 2020년 착공이 예정돼 있는 ...
4,당진시대,505279620,"['당진시정책자문위원회가 유명무실하다는 비판이 지속적으로 제기되고 있다.', '민선..."
...,...,...,...
9982,기호일보,745338220,['지지부진한 인천 내항 재개발사업의 물꼬를 터 줄 것으로 기대했던 도시재생 혁신지...
9983,광주일보,745367988,['청와대는 30일 문재인 대통령의 ‘1호 공약’인 고위공직자범죄수사처(공수처) 설...
9984,광주일보,745368130,"['광주지역 광공업 생산 감소율이 14개월 만에 최고를 기록했다.', '기아자동차 ..."
9985,광주일보,745368136,"['아름다운 가게 용봉점 헌책방이 개점 10년만에 문을 닫는 다.', '올해 열렸던..."


# Id와 Article_original 칼럼 추출(Main_df)

In [38]:
main_df = test.iloc[:,1:]

In [39]:
main_df.head()

,id,article_original
0,500733466,['▲ 석문간척지 임차법인협의회가 한국농어촌공사 당진지사 앞에 공공비축벼 320t을...
1,500742482,"['신 벌떼해장국이 손님들의 성원에 보답하고자 24시간 영업을 재개한다.', '또한..."
2,500742484,"['1년 전과 비교해 당진지역 휘발유 가격은 72원, 경유 가격은 103원 상승했다..."
3,504213810,['어기구 국회의원이 천연가스의 안정적 수급을 위해 2020년 착공이 예정돼 있는 ...
4,505279620,"['당진시정책자문위원회가 유명무실하다는 비판이 지속적으로 제기되고 있다.', '민선..."


In [40]:
main_df['article_original'][0]

"['▲ 석문간척지 임차법인협의회가 한국농어촌공사 당진지사 앞에 공공비축벼 320t을 쌓아두고 시위를 벌이고 있다.', '석문간척지 임차법인협의회(이하 간척지협의회)가 농림축산식품부의 부당한 간척지 임대료 책정에 반발하며 지난달 30일 한국농어촌공사 당진지사 앞에 공공비축벼 320t을 쌓고 시위를 벌였다.', '43개 영농조합법인이 소속된 간척지협의회는 이번 벼 야적 시위를 통해 현재 1kg당 2100원으로 책정된 임대료를 현재 쌀 판매가격인 1300원대로 인하할 것을 요구하고 있다.', '이들은 지난 12월 7일 농림축산식품부에 탄원서를 제출했지만 “임대료 인하는 올해 이후에나 가능하다”고 통보받은 상황이다.', '게다가 임차법인들의 계약기간이 올해 만료되기 때문에 임대료를 인하해도 지난 2년 동안의 손실 보상은 받을 수 없는 상황이다.', '이에 간척지협의회는 계약기간을 2년 연장하고, 연장된 기간 동안 인하된 임대료를 적용해 지난 2년 간의 손실에 대해 보상할 것을 제안했다.', '더불어 요구사항이 받아들여지지 않을 경우 벼 야적시위를 시작한 날짜인 지난해 12월 30일자로 임대료를 벼로 납부하겠다는 입장이다.', '김재용 봉치영농조합법인 조합원은 “현재 한국농어촌공사의 답변을 기다리고 있다”며 “상황을 지켜본 뒤 추가적인 야적 시위 여부를 결정할 계획”이라고 말했다.', '이어 “법을 만들고 집행할 때 현실성 있게 만들어야 한다”며 “농민이 정부를 믿을 수 있도록 조속히 해결책을 마련해주길 바란다”고 덧붙였다.']"

# 문서 추출 요약 함수

In [41]:
def summaring(content):
    lexrank = LexRank()
    
    content = content.replace(",",'\n') # 문장을 줄 단위로 나눔
    content = re.sub('[^가-힣a-zA-Z0-9\n\s]','',content)
#     content = content.replace("'",'')   # 각 문장이 ''로 돼있어 제거

    lexrank.summarize(content)          
    
    summaries = lexrank.probe(3)        # 3개의 문장으로 문서 요약
    
    summaries_f = '\n'.join(summaries)  
    
    return summaries_f

# Main_df에 함수 적용

In [48]:
main_df['article_original']= main_df['article_original'].map(summaring)
ㅇ

KeyError: 'article_original'

# 칼럼 이름 제출 형식에 맞게 변환

In [43]:
main_df.rename(columns={'article_original':'summary'},inplace=True)

In [44]:
main_df

,id,summary
0,500733466,43개 영농조합법인이 소속된 간척지협의회는 이번 벼 야적 시위를 통해 현재 1kg...
1,500742482,벌떼해장국의 주 메뉴는 뼈해장국과 감자탕이다\n 또한 벌떼해장국의 추천 메뉴는 콩...
2,500742484,한국석유공사가 운영하는 오피넷wwwopinetcokr에 따르면 최근 송악읍 휘발유...
3,504213810,이어 석탄화력발전과 원전에 의존하는 현재 정부의 전력 정책이 획기적으로 전환돼야 ...
4,505279620,정책자문위원으로 활동하고 있는 A씨는 연 1회 전체회의를 진행하는데 회의시간이 짧...
...,...,...
9982,745338220,보도에 따르면 최근 국토교통부 도시재생특별위원회 심의 결과 인천시가 신청한 내항1...
9983,745367988,청와대는 30일 문재인 대통령의 1호 공약인 고위공직자범죄수사처공수처 설치법안이 국...
9984,745368130,반도체 생산업체인 엠코테크놀로지코리아는 부가가치가 상대적으로 낮은 부품을 생산하면...
9985,745368136,광주시 북구 용봉동 아름다운 가게 용봉점 헌책방광주시 북구 설죽로 202 화인빌딩...


In [45]:
main_df['summary'][0]

' 43개 영농조합법인이 소속된 간척지협의회는 이번 벼 야적 시위를 통해 현재 1kg당 2100원으로 책정된 임대료를 현재 쌀 판매가격인 1300원대로 인하할 것을 요구하고 있다\n 게다가 임차법인들의 계약기간이 올해 만료되기 때문에 임대료를 인하해도 지난 2년 동안의 손실 보상은 받을 수 없는 상황이다\n 김재용 봉치영농조합법인 조합원은 현재 한국농어촌공사의 답변을 기다리고 있다며 상황을 지켜본 뒤 추가적인 야적 시위 여부를 결정할 계획이라고 말했다'

# csv형식으로 저장

In [46]:
main_df.to_csv('level1_v2.csv',index=False)

In [47]:
c = pd.read_csv('level1_v2.csv')
c

,id,summary
0,500733466,43개 영농조합법인이 소속된 간척지협의회는 이번 벼 야적 시위를 통해 현재 1kg...
1,500742482,벌떼해장국의 주 메뉴는 뼈해장국과 감자탕이다\n 또한 벌떼해장국의 추천 메뉴는 콩...
2,500742484,한국석유공사가 운영하는 오피넷wwwopinetcokr에 따르면 최근 송악읍 휘발유...
3,504213810,이어 석탄화력발전과 원전에 의존하는 현재 정부의 전력 정책이 획기적으로 전환돼야 ...
4,505279620,정책자문위원으로 활동하고 있는 A씨는 연 1회 전체회의를 진행하는데 회의시간이 짧...
...,...,...
9982,745338220,보도에 따르면 최근 국토교통부 도시재생특별위원회 심의 결과 인천시가 신청한 내항1...
9983,745367988,청와대는 30일 문재인 대통령의 1호 공약인 고위공직자범죄수사처공수처 설치법안이 국...
9984,745368130,반도체 생산업체인 엠코테크놀로지코리아는 부가가치가 상대적으로 낮은 부품을 생산하면...
9985,745368136,광주시 북구 용봉동 아름다운 가게 용봉점 헌책방광주시 북구 설죽로 202 화인빌딩...
